# Homework 9: Getting Familiar with NASA Polynomials
## Due Date:  Tuesday, November 7th at 11:59 PM

Read the NASA Polynomial dataset in raw format and parse and store the data into an .xml file.

### Review of the NASA Polynomials
You can find the NASA Polynomial file in `thermo.txt`.

You can find some details on the NASA Polynomials [at this site](http://combustion.berkeley.edu/gri_mech/data/nasa_plnm.html) in addition to the Lecture 16 notes.


The NASA polynomials for specie $i$ have the form:
$$
    \frac{C_{p,i}}{R}= a_{i1} + a_{i2} T + a_{i3} T^2 + a_{i4} T^3 + a_{i5} T^4
$$

$$
    \frac{H_{i}}{RT} = a_{i1} + a_{i2} \frac{T}{2} + a_{i3} \frac{T^2}{3} + a_{i4} \frac{T^3}{4} + a_{i5} \frac{T^4}{5} + \frac{a_{i6}}{T}
$$

$$
    \frac{S_{i}}{R}  = a_{i1} \ln(T) + a_{i2} T + a_{i3} \frac{T^2}{2} + a_{i4} \frac{T^3}{3} + a_{i5} \frac{T^4}{4} + a_{i7}
$$

where $a_{i1}$, $a_{i2}$, $a_{i3}$, $a_{i4}$, $a_{i5}$, $a_{i6}$, and $a_{i7}$ are the numerical coefficients supplied in NASA thermodynamic files. 

### Some Notes on `thermo.txt`
The first 7 numbers starting on the second line of each species entry (five of the second line and the first two of the third line) are the seven coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the high-temperature range (above 1000 K, the upper boundary is specified on the first line of the species entry). 

The next seven numbers are the coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the low-temperature range (below 1000 K, the lower boundary is specified on the first line of the species entry).

### Additional Specifications
Your final .xml file should contain the following specifications:

1. A `speciesArray` field that contains a space-separated list of all of the species present in the file.
2. Each species contains a `species` field with a `name` attribute as the species name.

    1. For each temperature range, use a sub-field with the minimum and maximum temperature as attributes.
    2. `floatArray` field that contains comma-separated string of each coefficient.
    
You can reference the `example_thermo.xml` file for an example .xml output.

**Hint**: First parse the file into a Python dictionary. 

In [2]:
def species_txt_to_dict(xml_file):
    file = open(xml_file,'r')
    lines = file.readlines()
    file.close()
    species_dict ={}
    Low = {}
    High = {}

    for i in lines:
        chars = i.split()
        #print(chars)
        if chars[-1]=='1':
            current_specie = chars[0]
            species_dict[current_specie]={'low':{},'high':{}}
            #Low Tmax Tmin
            Ltmin = chars[-4]
            Ltmax = chars[-2]
            Htmin = chars[-2]
            Htmax = chars[-3]
            #print(Ltmin,Ltmax,Htmin,Htmax)
        elif chars[-1] =='2':
            a1= i[0:15]
            a2= i[15:30]
            a3 = i[30:45]
            a4 = i[45:60]
            a5 = i[60:75]
            #print(i)
            #print(a1,a2,a3,a4,a5)
        elif chars[-1] == '3':
            a6 = i[0:15]
            a7 = i[15:30]

            coefs_highT = [a1,a2,a3,a4,a5,a6,a7]
            #for index,j in enumerate(coefs_highT):
            #    coefs_highT[index]=float(j)

            #print("coefs",coefs_highT)
            l1 = i[30:45]
            l2 = i[45:60]

        elif chars[-1] == '4':
            l3= i[0:15]
            l4= i[15:30]
            l5 = i[30:45]
            coefs_lowT = [l1,l2,l3,l4,l5]
            #for index,j in enumerate(coefs_lowT):
            #    coefs_lowT[index]=float(j)

            #print("low coef",coefs_lowT)
            species_dict[current_specie]['low']['coefs'] = coefs_lowT
            species_dict[current_specie]['low']['tmax'] = Ltmax
            species_dict[current_specie]['low']['tmin'] = Ltmin

            species_dict[current_specie]['high']['coefs'] = coefs_highT
            species_dict[current_specie]['high']['tmax'] = Htmax
            species_dict[current_specie]['high']['tmin'] = Htmin

            #print("current coefs",species_dict[current_specie]['high']['coefs'])
        if chars[0]=="END":
            #print("END")
            break
    
    return species_dict


def parse_dict_to_xml(species_dict):
    from copy import deepcopy
    import xml.etree.ElementTree as ET
    #use the template file
    tree = ET.parse('example_thermo.xml')
    root = tree.getroot()

    #get a node of species
    specie = deepcopy(root.find('speciesData').find('species'))

    #remove all the species from the parent
    old_species = root.find('speciesData').findall('species')

    for item in old_species:
        root.find('speciesData').remove(item)

    for index,key in enumerate(species_dict):

        temp = deepcopy(specie)
        temp.set('name',str(key))

        #species_dict[key]['low']['coefs']

        NASA = temp.find('thermo').findall('NASA')

        NASA[0].find('floatArray').text = ', '.join(species_dict[key]['low']['coefs'])
        NASA[0].set("Tmax",species_dict[key]['low']['tmax'])
        NASA[0].set("Tmin",species_dict[key]['low']['tmin'])

        NASA[1].find('floatArray').text = ', '.join(species_dict[key]['high']['coefs'])
        NASA[1].set("Tmax",species_dict[key]['high']['tmax'])
        NASA[1].set("Tmin",species_dict[key]['high']['tmin'])

        root.find('speciesData').append(temp)

    #creating my SpeciesArray
    for item in root.find('phase'):
        specie_names = [value for value in species_dict]
        item.text = " ".join(specie_names)

    return tree



In [3]:
my_species_dict = species_txt_to_dict('thermo_all.txt')
xml_tree = parse_dict_to_xml(my_species_dict)

xml_tree.write("NASAcoeffs.xml")

#pretify the resulting xml
import xml.dom.minidom
xml = xml.dom.minidom.parse("NASAcoeffs.xml")
pretty_xml_as_string = xml.toprettyxml()